In [29]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
table = pd.read_csv('my_table.csv')
table['date'] = table['date'].map(lambda x: dt.datetime.strptime(x.split('T')[0], '%Y-%m-%d'))
table.sort_values(['date','state'], inplace = True)
table.reset_index(drop=True, inplace=True)
table

,state,date,critical_staffing_shortage_today_yes,critical_staffing_shortage_today_no,critical_staffing_shortage_today_not_reported,critical_staffing_shortage_anticipated_within_week_yes,critical_staffing_shortage_anticipated_within_week_no,critical_staffing_shortage_anticipated_within_week_not_reported,hospital_onset_covid,hospital_onset_covid_coverage,...,staffed_pediatric_icu_bed_occupancy_coverage,total_staffed_pediatric_icu_beds,total_staffed_pediatric_icu_beds_coverage,staffed_icu_pediatric_patients_confirmed_covid,on_hand_supply_therapeutic_c_bamlanivimab_etesevimab_courses,previous_week_therapeutic_c_bamlanivimab_etesevimab_courses_used,previous_day_admission_pediatric_covid_confirmed_0_4,previous_day_admission_pediatric_covid_confirmed_unknown,previous_day_admission_pediatric_covid_confirmed_12_17,previous_day_admission_pediatric_covid_confirmed_5_11
0,AL,2020-01-01,0,0,1,0,0,1,NaN,0,...,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HI,2020-01-01,0,0,1,0,0,1,0.0,1,...,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IN,2020-01-01,0,0,1,0,0,1,0.0,1,...,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LA,2020-01-01,0,0,1,0,0,1,NaN,0,...,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MN,2020-01-01,0,0,1,0,0,1,0.0,1,...,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47796,VT,2022-08-08,3,13,1,6,10,1,1.0,16,...,16,4.0,16,0.0,148.0,0.0,0.0,0.0,0.0,0.0
47797,WA,2022-08-08,0,38,52,3,42,45,33.0,88,...,86,124.0,86,2.0,25.0,0.0,1.0,0.0,1.0,0.0
47798,WI,2022-08-08,20,95,24,19,96,24,12.0,136,...,136,98.0,136,0.0,3122.0,46.0,1.0,0.0,2.0,2.0
47799,WV,2022-08-08,0,8,54,24,34,4,28.0,58,...,58,112.0,58,8.0,31.0,0.0,2.0,0.0,2.0,0.0


**1 - ¿Cuáles fueron los 5 Estados con mayor ocupación hospitalaria por COVID? Criterio de ocupación por cama común. Considere la cantidad de camas ocupadas con pacientes confirmados y tome como referencia los 6 primeros meses del 2020 - recuerde incluir la cifra de infectados en esos meses (acumulativo). ¿Influye el rango etario en este comportamiento?**

Para resolver el problema, usaremos una tabla con los siguientes datos.
| fecha | estado | camas_usadas_por_covid |
|-|-|-|

Estaremos resolviendo con el **promedio** de pacientes en el día

In [3]:
fecha_inicio_01 = dt.datetime(2020,1,1) # Primero de enero del 2020
fecha_final_01 = dt.datetime(2020,6,30) # 30 de Junio del 2020
table01 = table[(table['date'] >= fecha_inicio_01) & (table['date'] <= fecha_final_01)].copy()
table01 = table01[['date', 'state', 'inpatient_beds_used_covid']]
table01.dropna(inplace=True)
table01.reset_index(drop=True, inplace=True)
table01

,date,state,inpatient_beds_used_covid
0,2020-01-01,HI,0.0
1,2020-01-01,IN,0.0
2,2020-01-01,MN,0.0
3,2020-01-01,MT,0.0
4,2020-01-01,NC,0.0
...,...,...,...
6605,2020-06-30,VT,10.0
6606,2020-06-30,WA,500.0
6607,2020-06-30,WI,364.0
6608,2020-06-30,WV,91.0


Vemos la cantidad de registros que hay para cada estado:

In [4]:
table01['state'].value_counts()

HI    182
MN    182
MT    182
NC    182
IN    182
TX    182
NV    164
KS    151
IL    134
MS    133
WV    132
MO    129
OR    128
CA    127
PR    127
WA    124
ND    122
WY    122
PA    122
OK    122
NJ    122
NE    122
OH    122
MI    122
ME    122
MD    122
LA    122
KY    122
IA    122
GA    122
AL    121
AZ    121
SC    121
VA    121
RI    120
WI    113
AR    112
FL    111
ID    110
CO    109
NM    109
NY    109
TN    107
CT    105
VT    104
UT    103
AK    100
SD     98
DE     96
MA     96
NH     96
DC     95
VI     84
Name: state, dtype: int64

Debido a que hay diferentes cantidades de registros para cada estado, lo conveniente es usar el promedio de pacientes en cama por día.

In [5]:
top_oc_hosp = table01[['state','inpatient_beds_used_covid']].groupby('state').mean().sort_values('inpatient_beds_used_covid', ascending=False)
top_oc_hosp.rename(columns={'inpatient_beds_used_covid':'Camas usadas por covid diariamente'}, inplace=True)
top_oc_hosp['Total en 6 meses'] = top_oc_hosp['Camas usadas por covid diariamente'] * (fecha_final_01 - fecha_inicio_01).days
top_oc_hosp.index.set_names('Estado', inplace=True)
top_oc_hosp.head()

,Camas usadas por covid diariamente,Total en 6 meses
Estado,,
NY,6298.422018,1.140014e+06
CA,3423.000000,6.195630e+05
FL,2902.945946,5.254332e+05
MA,2040.114583,3.692607e+05
IL,1597.089552,2.890732e+05


**2 - Analice la ocupación de camas (Común) por COVID en el Estado de Nueva York durante la cuarentena establecida e indique:**
* **Intervalos de crecimiento y decrecimiento**
* **Puntos críticos (mínimos y máximos)**

Para este problema usaremos como fecha inicial de cuarentena el _22 de marzo del 2020_ y fecha final el _13 de junio del 2020_. De esta manera la pandemia duró 84 días contando estos días límites. Fuentes: <br>
fecha inicial: https://edition.cnn.com/2020/03/20/politics/new-york-workforce-stay-home/index.html <br>
fecha final: https://www.cbsnews.com/news/new-york-stay-at-home-extended-coronavirus-lockdown/ 

In [6]:
fecha_inicio_02 = dt.datetime(2020,3,22) # 22 de marzo del 2020
fecha_final_02 = dt.datetime(2020,6,13) # 13 de junio del 2020
table02 = table[(table['date'] >= fecha_inicio_02) & (table['date'] <= fecha_final_02) & (table['state'] == 'NY')].copy()
table02 = table02[['date', 'inpatient_beds_used_covid']]
table02.reset_index(drop=True, inplace=True)
table02.rename(columns={'date':'Fecha', 'state':'Estado', 'inpatient_beds_used_covid':'Ocupación de camas por Covid en NY'}, inplace = True)
table02


,Fecha,Ocupación de camas por Covid en NY
0,2020-03-22,324.0
1,2020-03-23,417.0
2,2020-03-24,554.0
3,2020-03-25,843.0
4,2020-03-26,992.0
...,...,...
79,2020-06-09,3001.0
80,2020-06-10,2835.0
81,2020-06-11,2772.0
82,2020-06-12,2638.0


In [7]:
def add_criticos(table, columna):
    casos = np.array([])
    critico = np.array([])
    if table.loc[1,columna] - table.loc[0,columna] > 0:
        case = 'Creciente'
    else:
        case = 'Decreciente'
    for index in table.index:
        if index == 0 or index == table.index[-1]:
            casos = np.append(casos,case)
            if index == 0: critico = np.append(critico,'Limite')
            else: critico = np.append(critico,'Limite')
            continue
        actual_value = table.loc[index, columna]
        last_value = table.loc[index-1, columna]
        next_value = table.loc[index+1, columna]
        last_dif = actual_value - last_value
        next_dif = next_value - actual_value
        if last_dif*next_dif < 0 and last_dif < 0:
            critico = np.append(critico, 'Mínimo')
            casos = np.append(casos,'--')
            case = 'Creciente'
            continue
        elif last_dif*next_dif < 0:
            critico = np.append(critico, 'Máximo')
            casos = np.append(casos,'--')
            case = 'Decreciente'
            continue
        casos = np.append(casos,case)
        critico = np.append(critico, '--')
    table['Comportamiento'] = casos
    table['Criticos'] = critico   

In [8]:
add_criticos(table02, 'Ocupación de camas por Covid en NY')
table02

,Fecha,Ocupación de camas por Covid en NY,Comportamiento,Criticos
0,2020-03-22,324.0,Creciente,Limite
1,2020-03-23,417.0,Creciente,--
2,2020-03-24,554.0,Creciente,--
3,2020-03-25,843.0,Creciente,--
4,2020-03-26,992.0,Creciente,--
...,...,...,...,...
79,2020-06-09,3001.0,Decreciente,--
80,2020-06-10,2835.0,Decreciente,--
81,2020-06-11,2772.0,Decreciente,--
82,2020-06-12,2638.0,Decreciente,--


In [9]:
fig02 = go.Figure()
x0 = table02.loc[0,'Fecha']
comp = table02.loc[0,'Comportamiento']
for index in table02.index:
    if index == 0:
        continue
    elif table02.loc[index,'Comportamiento'] == '--':
        x1 = table02.loc[index,'Fecha']
        if comp == 'Creciente': color = 'red'
        else: color = 'green'
        fig02.add_vrect(x0,x1, fillcolor=color, line_width=0, opacity=0.2, annotation_text=comp[0], annotation_position="bottom left")
        x0 = table02.loc[index,'Fecha']
        if comp == 'Creciente': comp = 'Decreciente'
        else: comp = 'Creciente'
        continue
    elif index == table02.index[-1]:
        x1 = table02.loc[index,'Fecha']
        if comp == 'Creciente': color = 'red'
        else: color = 'green'
        fig02.add_vrect(x0,x1, fillcolor=color, line_width=0, opacity=0.2, annotation_text=comp[0], annotation_position="bottom left",)
        continue
fig02.add_trace(go.Scatter(x = table02['Fecha'], y = table02['Ocupación de camas por Covid en NY'], mode='lines', name='Número de camas'))
table02_ = table02[table02['Criticos'] == 'Máximo']
fig02.add_trace(go.Scatter(x = table02_['Fecha'], y = table02_['Ocupación de camas por Covid en NY'], mode='markers', name='Máximos'))
table02_ = table02[table02['Criticos'] == 'Mínimo']
fig02.add_trace(go.Scatter(x = table02_['Fecha'], y = table02_['Ocupación de camas por Covid en NY'], mode='markers', name='Mínimos'))
fig02.update_layout(
    title='Ocupación de camas por Covid en NY',
    xaxis_title = 'Fecha',
    yaxis_title = 'Número de camas')

Noticias relevantes
https://pix11.com/news/coronavirus/latest-coronavirus-updates-in-new-york-tuesday-april-14-2020/

**3 - ¿Cuáles fueron los cinco Estados que más camas UCI -Unidades de Cuidados Intensivos- utilizaron durante el año 2020? La medición debe realizarse en términos absolutos.**

In [10]:
fecha_inicio_03 = dt.datetime(2020,1,1) # 1ro de enero del 2020
fecha_final_03 = dt.datetime(2020,12,31) # 31 de diciembre del 2020
table03 = table[(table['date'] >= fecha_inicio_03) & (table['date'] <= fecha_final_03)].copy()
table03 = table03[['date', 'state', 'staffed_adult_icu_bed_occupancy']]
table03.dropna(inplace=True)
table03.reset_index(drop=True, inplace=True)
table03.rename(columns={'date':'Fecha', 'state':'Estado', 'staffed_adult_icu_bed_occupancy':'Ocupación de camas por adultos en UCI'}, inplace = True)
table03 = table03[['Estado', 'Ocupación de camas por adultos en UCI']].groupby('Estado').sum().sort_values('Ocupación de camas por adultos en UCI', ascending=False).reset_index()
table03.head()

,Estado,Ocupación de camas por adultos en UCI
0,TX,991193.0
1,CA,901654.0
2,FL,799387.0
3,NY,495372.0
4,PA,465537.0


In [11]:
num_states03 = 15
colors03 = ['rgb(179,179,179)']*num_states03
colors03[:5] = ['rgb(228,26,28)']*5
fig03 = go.Figure()
trace03 = go.Bar(x = table03.head(num_states03)['Estado'], 
                y = table03.head(num_states03)['Ocupación de camas por adultos en UCI'],
                marker_color = colors03, text=table03.head(num_states03)['Ocupación de camas por adultos en UCI'])
fig03.add_trace(trace03)
fig03.update_layout(title='Ocupación de camas por adultos en UCI en 2020')
fig03.update_traces(texttemplate='%{text:.3s}', textposition='auto')


**4 - ¿Qué cantidad de camas se utilizaron, por Estado, para pacientes pediátricos con COVID durante el 2020?**<br>
all_pediatric_inpatient_bed_occupied

In [19]:
fecha_inicio_04 = dt.datetime(2020,1,1) # 1ro de enero del 2020
fecha_final_04 = dt.datetime(2020,12,31) # 31 de diciembre del 2020
table04 = table[(table['date'] >= fecha_inicio_04) & (table['date'] <= fecha_final_04)].copy()
table04 = table04[['state', 'all_pediatric_inpatient_bed_occupied']]
table04.dropna(inplace=True)
table04.reset_index(drop=True, inplace=True)
table04.rename(columns={'state':'Estado', 'all_pediatric_inpatient_bed_occupied':'Camas ocupadas para pacientes pediátricos con COVID-19'}, inplace = True)
table04 = table04[['Estado', 'Camas ocupadas para pacientes pediátricos con COVID-19']].groupby('Estado').sum().sort_values('Camas ocupadas para pacientes pediátricos con COVID-19', ascending=False).reset_index()
fig = go.Figure()
fig.add_trace(go.Choropleth(locations=table04['Estado'],
                            locationmode='USA-states',
                            z = table04['Camas ocupadas para pacientes pediátricos con COVID-19'],
                            colorscale = 'Inferno_r',
                            colorbar_title = 'Camas para pacientes pediátricos',
                            marker_line_color='rgba(0,0,0,0)'))
fig.update_layout(title='Ocupación de camas para pacientes pediátricos',
                    geo_scope='usa', paper_bgcolor='rgba(0,0,0,0)', geo_bgcolor = 'rgba(0,0,0,0)',
                    autosize=False, margin = dict(l=0, r=0, b=0, t=25, pad=4, autoexpand=True))       

**5 - ¿Qué porcentaje de camas UCI corresponden a casos confirmados de COVID-19? Agrupe por Estado.**<br>
adult_icu_bed_covid_utilization

In [21]:
table05 = table.copy()
table05 = table05[['state', 'adult_icu_bed_covid_utilization']]
table05.dropna(inplace=True)
table05.reset_index(drop=True, inplace=True)
table05.rename(columns={'state':'Estado', 'adult_icu_bed_covid_utilization':'Porcentaje de camas UCI para covid'}, inplace = True)
table05 = table05[['Estado', 'Porcentaje de camas UCI para covid']].groupby('Estado').mean().sort_values('Porcentaje de camas UCI para covid', ascending=False).reset_index()
fig = go.Figure()
fig.add_trace(go.Choropleth(locations=table05['Estado'],
                            locationmode='USA-states',
                            z = table05['Porcentaje de camas UCI para covid'],
                            colorscale = 'Inferno_r',
                            colorbar_title = 'Camas para pacientes pedriáticos',
                            marker_line_color='rgba(0,0,0,0)'))
fig.update_layout(title='Porcentaje de camas UCI para covid por estado',
                    geo_scope='usa', paper_bgcolor='rgba(0,0,0,0)', geo_bgcolor = 'rgba(0,0,0,0)',
                    autosize=False, margin = dict(l=0, r=0, b=0, t=25, pad=4, autoexpand=True))       

**6 - ¿Cuántas muertes por covid hubo, por Estado, durante el año 2021?**<br>
deaths_covid

In [23]:
fecha_inicio_06 = dt.datetime(2021,1,1) # 1ro de enero del 2021
fecha_final_06 = dt.datetime(2021,12,31) # 31 de diciembre del 2021
table06 = table[(table['date'] >= fecha_inicio_06) & (table['date'] <= fecha_final_06)].copy()
table06 = table06[['state', 'deaths_covid']]
table06.dropna(inplace=True)
table06.rename(columns={'state':'Estado', 'deaths_covid':'Muertes por covid'}, inplace = True)
table06 = table06[['Estado', 'Muertes por covid']].groupby('Estado').sum().sort_values('Muertes por covid', ascending=False).reset_index()
fig = go.Figure()
fig.add_trace(go.Choropleth(locations=table06['Estado'],
                            locationmode='USA-states',
                            z = table06['Muertes por covid'],
                            colorscale = 'Inferno_r',
                            colorbar_title = 'Cantidad',
                            marker_line_color='rgba(0,0,0,0)'))
fig.update_layout(title='Muertes causadas por COVID-19',
                geo_scope='usa', paper_bgcolor='rgba(0,0,0,0)', geo_bgcolor = 'rgba(0,0,0,0)',
                autosize=False, margin = dict(l=0, r=0, b=0, t=25, pad=4, autoexpand=True))       

**7 - ¿Qué relación presenta la falta de personal médico, con la cantidad de muertes por covid durante el año 2021?**

In [28]:
table07 = table[['date', 'deaths_covid', 'critical_staffing_shortage_today_yes']].copy()
table07.dropna(inplace=True)
table07 = table07.groupby('date').sum().reset_index(drop = True)
min = 100
table_r = table07[table07['critical_staffing_shortage_today_yes'] >= min].copy()
fig = go.Figure()
trace07 = go.Scatter(y = table07['deaths_covid'],
                    x = table07['critical_staffing_shortage_today_yes'], mode = 'markers')
fig.add_trace(trace07)
fig.update_layout(
    title='Relación muertes por covid con falta de personal',
    yaxis_title = 'Muertes por Covid',
    xaxis_title = 'Número de hospitales con falta de personal')

In [31]:
X = table_r['critical_staffing_shortage_today_yes'].values.reshape(-1,1)
y = table_r['deaths_covid'].values
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.25, random_state=0)
lr = LinearRegression(fit_intercept=True)
lr.fit(X_tr, y_tr)
y_pr_tr = lr.predict(X_tr)
y_pr_te = lr.predict(X_te)
print('**Error en predicciones (MSE):** {}'.format(mean_squared_error(y_te,y_pr_te)))
fig = go.Figure()
fig.add_trace(go.Scatter(y = table_r['deaths_covid'], x = table_r['critical_staffing_shortage_today_yes'], mode = 'markers', name='Datos'))
fig.add_trace(go.Scatter(x = X_tr.reshape(-1), y = y_pr_tr, mode='markers', name='Train Data'))
fig.add_trace(go.Scatter(x = X_te.reshape(-1), y = y_pr_te, mode='markers', name='Test Data'))
fig.update_layout(
    title='Predicciones',
    xaxis_title = 'Falta de personal médico',
    yaxis_title = 'Muertes por COVID-19')

**Error en predicciones (MSE):** 123259.50762957764


In [32]:
X = table_r['critical_staffing_shortage_today_yes'].values
X = (X)**2
X = X.reshape(-1,1)
y = table_r['deaths_covid'].values
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.25, random_state=0)
lr = LinearRegression(fit_intercept=True)
lr.fit(X_tr, y_tr)
y_pr_tr = lr.predict(X_tr)
y_pr_te = lr.predict(X_te)
print('**Error en predicciones (MSE):** {}'.format(mean_squared_error(y_te,y_pr_te)))
fig = go.Figure()
X_tr = X_tr.reshape(-1)
X_tr = X_tr**0.5
X_te = X_te.reshape(-1)
X_te = X_te**0.5
fig.add_trace(go.Scatter(y = table_r['deaths_covid'], x = table_r['critical_staffing_shortage_today_yes'], mode = 'markers', name='Datos'))
fig.add_trace(go.Scatter(x = X_tr, y = y_pr_tr, mode='markers', name='Train Data'))
fig.add_trace(go.Scatter(x = X_te, y = y_pr_te, mode='markers', name='Test Data'))
fig.update_layout(
    title='Predicciones',
    xaxis_title = 'Falta de personal médico',
    yaxis_title = 'Muertes por COVID-19')

**Error en predicciones (MSE):** 114101.27364194232


Cuando aumenta la falta del personal médico, aumentan las muertes por covid

**8 - Siguiendo las respuestas anteriores, ¿cuál fue el peor mes de la pandemia para USA en su conjunto? Puede utilizar otras medidas que considere necesarias.**

In [17]:
table08 = table[['date', 'deaths_covid', 'inpatient_beds_used_covid', 'critical_staffing_shortage_today_yes']].copy()
table08['date'] = table08['date'].map(lambda x: dt.datetime.strptime(x.strftime('%Y-%m'), '%Y-%m'))
table08 = table08.groupby(['date']).sum().reset_index()
fig08 = go.Figure()
fig08.add_trace(go.Scatter(x = table08['date'], 
                         y = table08['deaths_covid']/ (table08['deaths_covid'].max()), 
                         name='Muertes por covid',
                         line=dict(color='rgba(228,26,28, .6)', width=2, dash='dash')))
fig08.add_trace(go.Scatter(x = table08['date'], 
                         y = table08['inpatient_beds_used_covid']/(table08['inpatient_beds_used_covid'].max()), 
                         name='Pacientes en cama con COVID-19',
                         line=dict(color='rgba(55, 126, 184, .6)', width=2, dash='dash')))
fig08.add_trace(go.Scatter(x = table08['date'], 
                         y = table08['critical_staffing_shortage_today_yes']/(table08['critical_staffing_shortage_today_yes'].max()), 
                         name='Número de hospitales con falta de personal',
                         line=dict(color='rgba(77, 175, 74, .6)', width=2, dash='dash')))
table08['prom_indicators'] = (table08['deaths_covid']/ (table08['deaths_covid'].max()) + table08['inpatient_beds_used_covid']/(table08['inpatient_beds_used_covid'].max()) + table08['critical_staffing_shortage_today_yes']/(table08['critical_staffing_shortage_today_yes'].max())) / 3
fig08.add_trace(go.Scatter(x = table08['date'], 
                         y = table08['prom_indicators']/table08['prom_indicators'].max(), 
                         name='Promedio de los 3 indicadores',
                         line=dict(color='rgba(0, 0, 0, 1)', width=3)))
fig08.update_layout(
    title='Muertes por COVID-19',
    xaxis_title = 'Fecha',
    yaxis_title = 'Porcentaje del valor máximo', plot_bgcolor='white')

**Enero del 2021 fue el pero mes**

**9 - ¿Qué recomendaciones haría, ex post, con respecto a los recursos hospitalarios y su uso?**

## Conclusiones
### * A lo largo de las exploraciones de los datos se puede concluir que los estados afecatos fueron Nueva York, California y Texas.
### * El peor momento en la pandemia ha sido el enero del 2021.
### * En enero de todos los años hay un repunte debido a la falta de personal enmes pasado y a las temperaturas frías.
### * Los gráficos coinciden en que los meses cálidos (sobre todo Junio y Julio) son mejores meses del año.
### * Se recomendaría preparar personal antes de las temperaturas frías para poabordar de mejor manera la pandemia. Esto debido a que la falta de personal médcrea tendencia a mayor número de muertes.
### * Aprovechar los climas cálidos para escatimar en gastos y poder usarlos en invierno.